# 0. 라이브러리 & 데이터

## 0.1 라이브러리

In [26]:
import pandas as pd
import numpy as np
import pickle as pk
from tqdm import tqdm

In [27]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.simplefilter("ignore")

## 0.2 데이터

In [28]:
case_fraud = pd.read_csv('../1_Data/case_fraud_prep.csv')
case_fraud.head(1)

,index,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,사건종류코드,판결유형,선고,판례상세링크,판시사항,판결요지,참조조문,참조판례,판례내용,판례내용_전처리,판례내용_전처리_한자,판례내용_전처리_글자수,판례내용_토큰화_코모란,판례내용_토큰화_꼬꼬마
0,20,238021,특정경제범죄가중처벌등에관한법률위반(사기)·사기,2023도12424,2023.11.16,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=yeorii&target=prec&ID=23...,"유사수신행위를 금지·처벌하는 유사수신행위의 규제에 관한 법률 제6조 제1항, 제3...","유사수신행위의 규제에 관한 법률(이하 ‘유사수신행위법’이라 한다) 제6조 제1항, ...","유사수신행위의 규제에 관한 법률 제3조, 제6조 제1항, 형법 제37조, 제347조...",내용없음,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 장용배【배상신청인】 배...,상고이유를 판단한다 불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 ...,상고이유를 판단한다. 1. 불가벌적 사후행위에 대한 판단「유사수신행위의 규제에...,1066,"['상고', '이유', '를', '판단', '하', 'ㄴ다', '불가벌적 사후행위'...","['상고', '이유', '를', '판단', '하', 'ㄴ다', '불가', '벌', ..."


In [29]:
kkmaVocab = []
with open("../2_Preprocessing/vocab_list/kkma.txt", "r") as file:
    for i in file:
        kkmaVocab.append(i.strip())
kkmaVocab[:10]

['상고', '이유', '판단', '불가', '사후', '행위', '판단', '유사', '수신', '행위']

In [30]:
# komoranVocab = []
# with open("../2_Preprocessing/vocab_list/kkma.txt", "r") as file:
#     for i in file:
#         komoranVocab.append(i.strip())
# komoranVocab[:10]

# 1. 학습

In [31]:
from gensim.models import FastText
model = FastText(kkmaVocab, vector_size=512, window=7, min_count=5, workers=4)

In [32]:
model.save('./model/fasttext.model')

# 2. 임베딩

In [33]:
model = FastText.load('./model/fasttext.model')

## 2.1 임베딩 전 전처리

### 2.1.1 이유 이후로 자르기

In [34]:
case_fraud['판례내용_전처리2'] = case_fraud.판례내용.str.split('유】|유]').str[1]

In [35]:
case_fraud.판례내용_전처리2[:2]

0      상고이유를 판단한다.  1. 불가벌적 사후행위에 대한 판단「유사수신행위의 규제에...
1      상고이유를 판단한다.   1. 피고인 1, 피고인 2, 피고인 3, 피고인 4,...
Name: 판례내용_전처리2, dtype: object

### 2.1.2 한자 전처리

In [36]:
import hanja
from hanja import hangul

def hanja_to_hangul(sent):
    return hanja.translate(sent, 'substitution')

In [37]:
case_fraud.판례내용_전처리2 = case_fraud.판례내용_전처리2.apply(hanja_to_hangul)

In [38]:
case_fraud.판례내용_전처리2[:2]

0      상고이유를 판단한다.  1. 불가벌적 사후행위에 대한 판단「유사수신행위의 규제에...
1      상고이유를 판단한다.   1. 피고인 1, 피고인 2, 피고인 3, 피고인 4,...
Name: 판례내용_전처리2, dtype: object

### 2.1.3 한글 외의 특수 문자 제거

In [39]:
import re

def remove_special_char(sent):
    sent = re.sub(r'\s[가-차]\.', '', sent)
    sent = re.sub(r'\s[가-차]\)', '', sent)
    sent = re.sub(r'다\)', '다.', sent)
    sent = re.sub(r'제\d+항', '', sent)
    sent = re.sub(r'제\d+조', '', sent)
    sent = re.sub(r'제\d+호', '', sent)
    sent = re.sub('[^가-힣. ]', '', sent)
    sent = re.sub(r'\. \.', '', sent)
    sent = re.sub(r'\.  \.', '.', sent)
    sent = re.sub(r'\.경', '', sent)
    return sent

In [40]:
case_fraud.판례내용_전처리2 = case_fraud.판례내용_전처리2.apply(remove_special_char)

In [41]:
case_fraud.판례내용_전처리2[0]

'  상고이유를 판단한다. 불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 법률이하 유사수신행위법이라 한다.   를 위반한 행위는 그 자체가 사기행위에 해당한다거나 사기행위를 반드시 포함한다고 할 수 없고 유사수신행위법 위반죄가 형법  의 사기죄와 구성요건을 달리하는 별개의 범죄로서 서로 보호법익이 다른 이상 유사수신행위를 한 자가 출자자에게 별도의 기망행위를 하여 유사수신행위로 조달받은 자금의 전부 또는 일부를 다시 투자받는 행위는 유사수신행위법 위반죄와 다른 새로운 보호법익을 침해하는 것으로서 유사수신행위법 위반죄의 불가벌적 사후행위가 되는 것이 아니라 별죄인 사기죄를 구성한다.  원심은 판시와 같은 이유로 피고인의   피해자 공소외인에 대한 판시 사기의 점에 관한 공소사실에 대하여 유사수신행위법 에서 금지하는 유사수신행위에는 기망행위가 포함되어 있지 않고 유사수신행위법 위반죄와 사기죄는 구성요건을 달리하는 별개의 범죄로서 서로 행위의 태양이나 보호법익을 달리한다는 등의 이유로 기존 범죄인 유사수신행위법 위반죄의 가벌적 평가범위 내에 흡수되는 불가벌적 사후행위에 해당하지 않는다고 판단하였다.  원심판결 이유를 위 법리 및 기록에 비추어 살펴보면 이 부분 원심판단에 일부 적절하지 않은 점이 있으나 피고인의   피해자 공소외인에 대한 판시 사기죄가 유사수신행위법 위반죄의 불가벌적 사후행위에 해당하지 않는다고 본 원심의 판단에 불가벌적 사후행위에 관한 법리를 오해함으로써 판결에 영향을 미친 잘못이 없다. 나머지 상고이유에 대한 판단  이 부분 상고이유는 형사소송법  에서 정한 형보다 가벼운 형이 선고된 이 사건에서 사실오인법리오해를 내세우며 실질적으로 원심의 증거 선택 및 증명력에 관한 판단 또는 이에 기초한 사실인정을 탓하는 것이거나 원심이 인정한 사실과 다른 사실관계를 전제로 법리오해를 지적하는 취지의 주장 또는 형이 너무 무거워 부당하다는 취지의 주장에 해당하여 모두 적법한 상고이유로 볼 수 없다. 결론  그러므로 상고를 기각하기로 하여 관여 대법관

In [43]:
case_fraud_sent = pd.DataFrame(case_fraud[['판례일련번호', '판례내용_전처리2']])
case_fraud_sent.head()

,판례일련번호,판례내용_전처리2
0,238021,상고이유를 판단한다. 불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 법...
1,237943,상고이유를 판단한다. . 피고인 피고인 피고인 피고인 피고인 에 대한...
2,236477,직권으로 판단한다. . 의료법 위반의 점에 관한 공소사실의 요지 피고인은...
3,236823,상고이유를 판단한다. 사건의 개요와 쟁점 의료법 위반의 점에 관한 공소사실의...
4,237903,. 이 법원의 심판범위 원심은 배상신청인들의 배상신청을 모두 각하하였다. 소송...


In [44]:
case_fraud_sent.to_csv('../1_Data/case_fraud_sent.csv')

## 2.2 문장 토큰화

In [157]:
# import kss

# sentence_tokened = []

# kss.split_sentences(case_fraud.판례내용_전처리2[0])

# # for x in tqdm(case_fraud['판례내용_전처리2']):
# #     sentence_tokened.append(kss.split_sentences(x))

['상고이유를 판단한다.',
 '불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 법률이하 유사수신행위법이라 한다.',
 '를 위반한 행위는 그 자체가 사기행위에 해당한다거나 사기행위를 반드시 포함한다고 할 수 없고 유사수신행위법 위반죄가 형법  의 사기죄와 구성요건을 달리하는 별개의 범죄로서 서로 보호법익이 다른 이상 유사수신행위를 한 자가 출자자에게 별도의 기망행위를 하여 유사수신행위로 조달받은 자금의 전부 또는 일부를 다시 투자받는 행위는 유사수신행위법 위반죄와 다른 새로운 보호법익을 침해하는 것으로서 유사수신행위법 위반죄의 불가벌적 사후행위가 되는 것이 아니라 별죄인 사기죄를 구성한다.',
 '원심은 판시와 같은 이유로 피고인의   피해자 공소외인에 대한 판시 사기의 점에 관한 공소사실에 대하여 유사수신행위법 에서 금지하는 유사수신행위에는 기망행위가 포함되어 있지 않고 유사수신행위법 위반죄와 사기죄는 구성요건을 달리하는 별개의 범죄로서 서로 행위의 태양이나 보호법익을 달리한다는 등의 이유로 기존 범죄인 유사수신행위법 위반죄의 가벌적 평가범위 내에 흡수되는 불가벌적 사후행위에 해당하지 않는다고 판단하였다.',
 '원심판결 이유를 위 법리 및 기록에 비추어 살펴보면 이 부분 원심판단에 일부 적절하지 않은 점이 있으나 피고인의   피해자 공소외인에 대한 판시 사기죄가 유사수신행위법 위반죄의 불가벌적 사후행위에 해당하지 않는다고 본 원심의 판단에 불가벌적 사후행위에 관한 법리를 오해함으로써 판결에 영향을 미친 잘못이 없다.',
 '나머지 상고이유에 대한 판단  이 부분 상고이유는 형사소송법  에서 정한 형보다 가벼운 형이 선고된 이 사건에서 사실오인법리오해를 내세우며 실질적으로 원심의 증거 선택 및 증명력에 관한 판단 또는 이에 기초한 사실인정을 탓하는 것이거나 원심이 인정한 사실과 다른 사실관계를 전제로 법리오해를 지적하는 취지의 주장 또는 형이 너무 무거워 부당하다는 취지의 주장에 해당하여 모두 적법한 상고이유로 볼 수 없다.',
 '결론  그러므로 

In [159]:
from nltk import sent_tokenize

sentence_tokened = []

for x in tqdm(case_fraud['판례내용_전처리2']):
    sentence_tokened.append(sent_tokenize(x))

100%|██████████| 2253/2253 [00:00<00:00, 3263.28it/s]


In [160]:
sentence_tokened[0]

['  상고이유를 판단한다.',
 '불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 법률이하 유사수신행위법이라 한다.',
 '를 위반한 행위는 그 자체가 사기행위에 해당한다거나 사기행위를 반드시 포함한다고 할 수 없고 유사수신행위법 위반죄가 형법  의 사기죄와 구성요건을 달리하는 별개의 범죄로서 서로 보호법익이 다른 이상 유사수신행위를 한 자가 출자자에게 별도의 기망행위를 하여 유사수신행위로 조달받은 자금의 전부 또는 일부를 다시 투자받는 행위는 유사수신행위법 위반죄와 다른 새로운 보호법익을 침해하는 것으로서 유사수신행위법 위반죄의 불가벌적 사후행위가 되는 것이 아니라 별죄인 사기죄를 구성한다.',
 '원심은 판시와 같은 이유로 피고인의   피해자 공소외인에 대한 판시 사기의 점에 관한 공소사실에 대하여 유사수신행위법 에서 금지하는 유사수신행위에는 기망행위가 포함되어 있지 않고 유사수신행위법 위반죄와 사기죄는 구성요건을 달리하는 별개의 범죄로서 서로 행위의 태양이나 보호법익을 달리한다는 등의 이유로 기존 범죄인 유사수신행위법 위반죄의 가벌적 평가범위 내에 흡수되는 불가벌적 사후행위에 해당하지 않는다고 판단하였다.',
 '원심판결 이유를 위 법리 및 기록에 비추어 살펴보면 이 부분 원심판단에 일부 적절하지 않은 점이 있으나 피고인의   피해자 공소외인에 대한 판시 사기죄가 유사수신행위법 위반죄의 불가벌적 사후행위에 해당하지 않는다고 본 원심의 판단에 불가벌적 사후행위에 관한 법리를 오해함으로써 판결에 영향을 미친 잘못이 없다.',
 '나머지 상고이유에 대한 판단  이 부분 상고이유는 형사소송법  에서 정한 형보다 가벼운 형이 선고된 이 사건에서 사실오인법리오해를 내세우며 실질적으로 원심의 증거 선택 및 증명력에 관한 판단 또는 이에 기초한 사실인정을 탓하는 것이거나 원심이 인정한 사실과 다른 사실관계를 전제로 법리오해를 지적하는 취지의 주장 또는 형이 너무 무거워 부당하다는 취지의 주장에 해당하여 모두 적법한 상고이유로 볼 수 없다.',
 '결론  그러므

In [161]:
embeddings = []

for sentences in tqdm(sentence_tokened):
    embedding = None
    count = 0
    for sentence in sentences:
        if embedding is None:
            embedding = model.wv.get_sentence_vector(sentence)
        else:
            embedding += model.wv.get_sentence_vector(sentence)
        count += 1
    embeddings.append(embedding / count)

100%|██████████| 2253/2253 [00:42<00:00, 53.43it/s] 


In [163]:
with open("./model/fasttext_emb.pickle", "wb") as fw:
    pk.dump(embeddings, fw)